# Models

In [2]:
from langchain.chat_models import init_chat_model

model = init_chat_model("openai:gpt-5-nano")
response = model.invoke("Why do parrots talk?")

print(response)

content='Parrots talk mainly because they are natural vocal learners and highly social birds.\n\n- Biology: Their brains and their special voice box (the syrinx) let them imitate a wide range of sounds. Some species are especially good at learning new noises and words.\n\n- Social reasons: In the wild, parrots use sounds to stay in touch with their flock, signal danger, attract mates, and coordinate activities. Mimicking other noises is part of how they fit into their social group.\n\n- In captivity: When kept as pets, they often imitate human speech to get attention, companionship, or a reward. If people talk to them a lot or reward words, they’ll repeat those words more.\n\n- Not all parrots talk: Some species and individuals talk more than others, and some may never learn to speak. A few parrots can use words in useful ways or show understanding of simple concepts, but many just imitate.\n\n- About understanding: They may not “mean” the words like humans do, but they can learn which

## 模型传入的参数

- model
- api_key
- temperature
- max_tokens
- max-retries



## 1. 调用

In [ ]:
response = model.invoke("Why do parrots have colorful feathers?")
print(response)

In [ ]:
from langchain.messages import HumanMessage, AIMessage, SystemMessage

conversation = [
    SystemMessage("You are a helpful assistant that translates English to French."),
    HumanMessage("Translate: I love programming."),
    AIMessage("J'adore la programmation."),
    HumanMessage("Translate: I love building applications.")
]

response = model.invoke(conversation)
print(response)  # AIMessage("J'adore créer des applications.")

### 流式传输

In [ ]:
for chunk in model.stream("Why do parrots have colorful feathers?"):
    print(chunk.text, end="|", flush=True)

In [ ]:
full = None  # None | AIMessageChunk
for chunk in model.stream("What color is the sky?"):
    full = chunk if full is None else full + chunk
    print(full.text)

# The
# The sky
# The sky is
# The sky is typically
# The sky is typically blue
# ...

print(full.content)
# Usually blue during the day. That blue comes from Rayleigh scattering: sunlight hits the atmosphere and the shorter blue wavelengths scatter in all directions. The color can change: red/orange/pink at sunrise and sunset, gray on cloudy days, and dark at night (with stars visible). Want the answer for a specific time or place?

### 批量 Batch

In [ ]:
# 返回整个批次的最终输出
responses = model.batch([
    "Why do parrots have colorful feathers?",
    "How do airplanes fly?",
    "What is quantum computing?"
])
for response in responses:
    print(response.content)

In [ ]:
# 逐个接收每个输入生成完成时的输出.在使用 batch_as_completed() 时，结果可能会乱序到达。每个结果都包含输入索引，以便在需要时匹配以重建原始顺序。
for response in model.batch_as_completed([
    "Why do parrots have colorful feathers?",
    "How do airplanes fly?",
    "What is quantum computing?"
]):
    print(response)

## 2.工具调用

In [ ]:
from langchain_core.tools import tool

@tool
def get_weather(location: str) -> str:
    """Get the weather at a location."""
    return f"It's sunny in {location}."


model_with_tools = model.bind_tools([get_weather])

response = model_with_tools.invoke("What's the weather like in Boston?")

for tool_call in response.tool_calls:
    # View tool calls made by the model
    print(f"Tool: {tool_call['name']}")
    print(f"Args: {tool_call['args']}")

## 2.1 工具执行循环

In [ ]:
# 将（可能多个）工具绑定到模型
model_with_tools = model.bind_tools([get_weather])

# Step 1: 模型生成工具调用
messages = [{"role": "user", "content": "What's the weather in Boston?"}]
ai_msg = model_with_tools.invoke(messages)
messages.append(ai_msg)

# Step 2: 执行工具并收集结果
for tool_call in ai_msg.tool_calls:
    # 使用生成的参数执行该工具
    tool_result = get_weather.invoke(tool_call)
    messages.append(tool_result)

# Step 3: 将结果传回模型以获得最终响应
final_response = model_with_tools.invoke(messages)
print(final_response.text)
# "The current weather in Boston is 72°F and sunny."

2.2 强制工具调用

强制使用任何工具： 
```python
model_with_tools = model.bind_tools([tool_1], tool_choice="any")
```

强制使用特定工具
```python
model_with_tools = model.bind_tools([tool_1], tool_choice="tool_1")
```

### 2.3 并行工具调用


In [ ]:
model_with_tools = model.bind_tools([get_weather])

response = model_with_tools.invoke(
    "What's the weather in Boston and Tokyo?"
)


# The model may generate multiple tool calls
print(response.tool_calls)
# [
#   {'name': 'get_weather', 'args': {'location': 'Boston'}, 'id': 'call_1'},
#   {'name': 'get_time', 'args': {'location': 'Tokyo'}, 'id': 'call_2'}
# ]


# Execute all tools (can be done in parallel with async)
results = []
for tool_call in response.tool_calls:
    if tool_call['name'] == 'get_weather':
        result = get_weather.invoke(tool_call)
    # ...
    results.append(result)
    
print( results)

### 2.4 流式工具调用

In [ ]:
for chunk in model_with_tools.stream(
    "What's the weather in Boston and Tokyo?"
):
    # Tool call chunks arrive progressively
    if chunk.tool_call_chunks:
        for tool_chunk in chunk.tool_call_chunks:
            print(f"Tool: {tool_chunk.get('name', '')}")
            print(f"Args: {tool_chunk.get('args', '')}")

# Output:
# Tool: get_weather            # Loop 1
# Args:
# Tool:                        # Loop 2
# Args: {"loc
# Tool:                        # Loop 3
# Args: ation": "BOS"}
# Tool: get_time               # Loop 4
# Args:
# Args:
# Tool:                        # Loop 5
# Args: {"timezone": "Tokyo"}

In [ ]:
gathered = None
for chunk in model_with_tools.stream("What's the weather in Boston?"):
    gathered = chunk if gathered is None else gathered + chunk
    print(gathered)

## 3.结构树输出

In [3]:
from pydantic import BaseModel, Field

class Movie(BaseModel):
    """A movie with details."""
    title: str = Field(..., description="The title of the movie")
    year: int = Field(..., description="The year the movie was released")
    director: str = Field(..., description="The director of the movie")
    rating: float = Field(..., description="The movie's rating out of 10")

model_with_structure = model.with_structured_output(Movie)
response = model_with_structure.invoke("Provide details about the movie Inception")
print(response)  # Movie(title="Inception", year=2010, director="Christopher Nolan", rating=8.8)

title='Inception' year=2010 director='Christopher Nolan' rating=8.8


## 4. 高级配置 - 多模态

In [5]:
response = model.invoke("Create a picture of a cat")
print(response.content)

Sure! Here are a couple of simple ASCII cats. Tell me if you want a different style.

Option 1 (tiny and cute):
 /\
/  \_/\ 
(=^.^=)
 (")(")

Option 2 (slightly bigger):
 /\_/\  
( o.o ) 
 > ^ <

If you’d prefer a real image or a different style (pixel art, line drawing, etc.), I can describe or generate that too.


## 4.1 高级配置 - 推理

In [9]:
for chunk in model.stream("Why do parrots have colorful feathers?"):
    print(chunk.content)


Par
rots
 are
 colorful
 mainly
 because
 of
 two
 things
:
 pigments
 in
 their
 feathers
 and
 the
 tiny
 structures
 inside
 the
 feathers
 that
 bend
 light
.


-
 Pig
ments
:
 

 
 -
 Car
ot
eno
ids
 (
from
 the
 birds
’
 diet
 of
 fruits
,
 seeds
,
 and
 plants
)
 give
 reds
,
 oranges
,
 and
 yell
ows
.

 
 -
 Mel
an
ins
 give
 brow
ns
 and
 blacks
.

 
 -
 Green
 is
 often
 yellow
 pigment
 plus
 a
 blue
 structural
 effect
,
 not
 a
 true
 green
 pigment
.

-
 Feather
 structure
:
 

 
 -
 Some
 blues
 and
 greens
 come
 from
 microscopic
 arrangements
 in
 the
 feather
 that
 reflect
 certain
 wavelengths
 of
 light
 (
not
 from
 pigments
).

 
 -
 Ir
ides
cence
 (
sh
immer
ing
,
 changing
 colors
)
 comes
 from
 layered
 structures
 in
 the
 feathers
.


Why
 they
’re
 colorful
:

-
 M
ating
 and
 social
 signals
:
 bright
 plum
age
 can
 show
 a
 bird
 is
 healthy
 and
 good
 at
 finding
 food
,
 helping
 attract
 mates
 and
 deter
 rivals
.

-
 Species
 and
 individual
 r

## 4.3 高级配置 - 缓存

In [12]:
from langchain_core.caches import InMemoryCache
from langchain_core.globals import set_llm_cache

set_llm_cache(InMemoryCache())

response = model.invoke("Tell me a joke")
print(response)
response = model.invoke("Tell me a joke")  # Fast, from cache
print(response)

content="Why don't skeletons fight each other? They don't have the guts. Want to hear another?" additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 413, 'prompt_tokens': 10, 'total_tokens': 423, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 384, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-5-nano-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-CKawUH2GOINN50SfnqEfICXZDje6M', 'service_tier': None, 'finish_reason': 'stop', 'logprobs': None} id='run--8ad8fd34-f923-4d0e-beff-458e452bb600-0' usage_metadata={'input_tokens': 10, 'output_tokens': 413, 'total_tokens': 423, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 384}}
content="Why don't skeletons fight each other? They don't have the guts. Want to hear another?" additional_kwargs={'refusal': None} respons

## 4.4 高级配置 - 速率限制

In [13]:
from langchain_core.rate_limiters import InMemoryRateLimiter

rate_limiter = InMemoryRateLimiter(
        requests_per_second=0.1,  # 表示每秒只能发 0.1 次请求，等价于 10 秒允许 1 次请求。
        check_every_n_seconds=0.1,  # 限流器会每 0.1 秒（100 毫秒）检查一次，看看是不是可以放行请求
        max_bucket_size=10,  # 采用令牌桶算法（token bucket）。允许一次性“积攒”最多 10 个请求作为突发流量。例如你空闲了 100 秒没请求，就能一次性发 10 个，但不会无限积累。
)

model = init_chat_model(
    model="gpt-5",
    model_provider="openai",
    rate_limiter=rate_limiter
)

## 4.5 高级配置 - 对数概率

In [14]:
model = init_chat_model(
    model="gpt-4o",
    model_provider="openai"
).bind(logprobs=True)

response = model.invoke("Why do parrots talk?")
print(response.response_metadata["logprobs"])

{'content': [{'token': 'Par', 'bytes': [80, 97, 114], 'logprob': -0.00010024999, 'top_logprobs': []}, {'token': 'rots', 'bytes': [114, 111, 116, 115], 'logprob': -1.1920922e-06, 'top_logprobs': []}, {'token': ' are', 'bytes': [32, 97, 114, 101], 'logprob': -0.84335274, 'top_logprobs': []}, {'token': ' able', 'bytes': [32, 97, 98, 108, 101], 'logprob': -2.1476667, 'top_logprobs': []}, {'token': ' to', 'bytes': [32, 116, 111], 'logprob': -9.536739e-07, 'top_logprobs': []}, {'token': ' "', 'bytes': [32, 34], 'logprob': -0.07432972, 'top_logprobs': []}, {'token': 'talk', 'bytes': [116, 97, 108, 107], 'logprob': -5.960463e-07, 'top_logprobs': []}, {'token': '"', 'bytes': [34], 'logprob': -0.03809879, 'top_logprobs': []}, {'token': ' or', 'bytes': [32, 111, 114], 'logprob': -0.9490852, 'top_logprobs': []}, {'token': ' mimic', 'bytes': [32, 109, 105, 109, 105, 99], 'logprob': -0.009784595, 'top_logprobs': []}, {'token': ' human', 'bytes': [32, 104, 117, 109, 97, 110], 'logprob': -0.011726613,

## 4.6 高级配置 - 令牌使用

In [16]:
from langchain.chat_models import init_chat_model
from langchain_core.callbacks import UsageMetadataCallbackHandler

llm_1 = init_chat_model(model="openai:gpt-4o-mini")

callback = UsageMetadataCallbackHandler()
result_1 = llm_1.invoke("Hello", config={"callbacks": [callback]})
callback.usage_metadata

{'gpt-4o-mini-2024-07-18': {'input_tokens': 8,
  'output_tokens': 10,
  'total_tokens': 18,
  'input_token_details': {'audio': 0, 'cache_read': 0},
  'output_token_details': {'audio': 0, 'reasoning': 0}}}

## 4.7 高级配置 - 调用配置

In [18]:
response = model.invoke(
    "Tell me a joke",
        config={
        "run_name": "joke_generation",      # Custom name for this run
        "tags": ["humor", "demo"],          # Tags for categorization
        "metadata": {"user_id": "123"},     # Custom metadata
    }
)

## 4.8 高级配置 - 可配置模型

In [21]:
from langchain.chat_models import init_chat_model

configurable_model = init_chat_model(temperature=0)

configurable_model.invoke(
    "what's your name",
        config={"configurable": {"model": "gpt-4o-mini"}},  # Run with GPT-5-Nano
)

AIMessage(content='I’m called ChatGPT. How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 14, 'prompt_tokens': 11, 'total_tokens': 25, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_efad92c60b', 'id': 'chatcmpl-CKblnGfQDgIpjmDEcN9KjVHFztYBr', 'service_tier': None, 'finish_reason': 'stop', 'logprobs': None}, id='run--068ba088-19dc-4794-81aa-5ff90f5d0cc7-0', usage_metadata={'input_tokens': 11, 'output_tokens': 14, 'total_tokens': 25, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})